In [1]:
import subprocess
from io import BytesIO

import folium
import hvplot.pandas
import pandas as pd
import requests

In [2]:
sg_lat = 46.08666667
sg_lon = -115.5138889

# Initialize map and tweak settings
m = folium.Map(
    # Location to display
    location=(sg_lat, sg_lon),
    # Turns off annoying zooming while trying to scroll to the next cell
    scrollWheelZoom=False)

# Put a marker at the stream gauge location
folium.Marker([sg_lat, sg_lon], popup="Thingy").add_to(m)

# Display the map
m

## Site Information
The data was collected from a stream guage placed on the Selway River in central idaho in the Nez Perce-Clearwater National Forest. This is a mountainous and sparcely populated region of Idaho which is the known for its largely untouched river corridors and mountain ecosystems. 

In [3]:
# URL for USGS NWIS data from Omaha NE
selway_url = ("https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb"
             "&site_no=13336500&legacy=&referred_module=sw&period=&begin"
             "_date=1923-09-01&end_date=2023-09-01")

# HTTP data request
raw_selway_data = requests.get(selway_url)

raw_selway_data

<Response [200]>

In [4]:
# Take a look at the data. What got downloaded?
for i, line in enumerate(raw_selway_data.content.splitlines()[:35]):
    # Skip commented lines
    if not line.startswith(b'#'):
        print(i, line)

29 b'agency_cd\tsite_no\tdatetime\t326226_00060_00003\t326226_00060_00003_cd'
30 b'5s\t15s\t20d\t14n\t10s'
31 b'USGS\t13336500\t1929-10-01\t520\tA'
32 b'USGS\t13336500\t1929-10-02\t520\tA'
33 b'USGS\t13336500\t1929-10-03\t520\tA'
34 b'USGS\t13336500\t1929-10-04\t520\tA'


In [5]:
selway_q_df = pd.read_csv(
    BytesIO(raw_selway_data.content),
    comment='#',
    delimiter='\t',
    skiprows=[29, 30],
    names=['agency_cd', 'site_no', 'datetime', 'streamflow_cfs', 'code'],
    index_col='datetime',
    parse_dates=True,
    
)

selway_q_df

,agency_cd,site_no,streamflow_cfs,code
datetime,,,,
1929-10-01,USGS,13336500,520.0,A
1929-10-02,USGS,13336500,520.0,A
1929-10-03,USGS,13336500,520.0,A
1929-10-04,USGS,13336500,520.0,A
1929-10-05,USGS,13336500,520.0,A
...,...,...,...,...
2023-08-28,USGS,13336500,601.0,P
2023-08-29,USGS,13336500,599.0,P
2023-08-30,USGS,13336500,709.0,P


In [6]:
selway_q_sub_df = selway_q_df['1947-10':'1950-09']

selway_q_sub_df

,agency_cd,site_no,streamflow_cfs,code
datetime,,,,
1947-10-01,USGS,13336500,907.0,A
1947-10-02,USGS,13336500,916.0,A
1947-10-03,USGS,13336500,2140.0,A
1947-10-04,USGS,13336500,2480.0,A
1947-10-05,USGS,13336500,1520.0,A
...,...,...,...,...
1950-09-26,USGS,13336500,1080.0,A
1950-09-27,USGS,13336500,1350.0,A
1950-09-28,USGS,13336500,1100.0,A


In [7]:
selway_q_plot = selway_q_sub_df.hvplot.line(x='datetime', y='streamflow_cfs')

selway_q_plot

/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [datetime]   (streamflow_cfs)

In [8]:
selway_streamflow_df = selway_q_df[['streamflow_cfs']]

selway_year_am_df = selway_streamflow_df.resample('AS').max()

selway_year_am_df

,streamflow_cfs
datetime,
1929-01-01,2470.0
1930-01-01,14600.0
1931-01-01,16400.0
1932-01-01,27900.0
1933-01-01,31200.0
...,...
2019-01-01,25800.0
2020-01-01,28100.0
2021-01-01,20900.0


# Selway River streamflows have decreased on average since the record high flows in May 1948

In [9]:
selway_year_am_df.hvplot()

/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [datetime]   (streamflow_cfs)

## Data Source
This data was collected by the US Geological Survey and was requested from the
National Water information System online platform. It provided the historical records from the stream guage On the Selway River near Lowell Idaho. 2. National Water Information System, USGS.https://dashboard.waterdata.usgs.gov/app/nwd/en/?aoi%253Ddefault

In [10]:
%%capture
%%bash
jupyter nbconvert flood-return-selway-river.ipynb --to html --no-input